<a href="https://colab.research.google.com/github/RafsanJany-44/Thesis_Project/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

#Data Set

In [ ]:
!pip install tensorflow_datasets

#GPU setup

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:

tf.config.list_physical_devices("GPU")
tf.test.gpu_device_name()
# Standard output is '/device:GPU:0'
gpus = tf.config.list_physical_devices("GPU")

#tf.config.experimental.set_memory_growth(gpus[0],True)

#tf.config.experimental.set_visible_devices(gpus[0],'GPU')
#logical_gpus= tf.config.experimental.list_logical_devices('GPU')
#print(len(gpus),"Physical GPUs", len(logical_gpus),"Logical GPU")

#Available Data set

In [ ]:
tfds.list_builders()

['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'arc',
 'bair_robot_pushing_small',
 'bccd',
 'beans',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'blimp',
 'bool_q',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clic',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coco_captions',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'coqa',
 'cos_e',
 'cosmos_qa',
 'covid19sum',
 'crema_d',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'deep_weeds',
 'definite_pronoun_resolution',
 'dementiabank',
 'diabetic_retinopathy_detection',
 'div2k',
 'dmlab',
 'downsampled_imagenet',
 

In [ ]:
tfds.builder("imdb_reviews").info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [ ]:
imdb_train,ds_info=tfds.load(name="imdb_reviews",split="train",with_info=True,as_supervised=True)
imdb_test = tfds.load(name="imdb_reviews",split="test",as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteCEN9ZV/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteCEN9ZV/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
type(imdb_train)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [ ]:
for example,label in imdb_train.take(3):
  print(example,"\n",label)

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string) 
 tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on

In [ ]:
tokenizer = tfds.deprecated.text.Tokenizer()
vocabulary_set = set()

for example,label in imdb_train:
  some_token=tokenizer.tokenize(example.numpy())
  vocabulary_set.update(some_token)

In [ ]:
some_token

['First',
 'of',
 'all',
 'Riget',
 'is',
 'wonderful',
 'Good',
 'comedy',
 'and',
 'mystery',
 'thriller',
 'at',
 'the',
 'same',
 'time',
 'Nice',
 'combination',
 'of',
 'strange',
 'dogma',
 'style',
 'of',
 'telling',
 'the',
 'story',
 'together',
 'with',
 'good',
 'music',
 'and',
 'great',
 'actors',
 'But',
 'unfortunately',
 'there',
 's',
 'no',
 'the',
 'end',
 'As',
 'for',
 'me',
 'it',
 's',
 'unacceptable',
 'I',
 'was',
 'thinking',
 'how',
 'it',
 'will',
 'be',
 'possible',
 'to',
 'continue',
 'the',
 'story',
 'without',
 'Helmer',
 'and',
 'Drusse',
 'and',
 'I',
 'have',
 'some',
 'idea',
 'I',
 'think',
 'Lars',
 'should',
 'make',
 'RIGET',
 'III',
 'a',
 'little',
 'bit',
 'different',
 'I',
 'm',
 'sure',
 'that',
 '3rd',
 'part',
 'without',
 'Helmer',
 'wouldn',
 't',
 'be',
 'the',
 'same',
 'So',
 'here',
 's',
 'my',
 'suggestion',
 'Mayble',
 'little',
 'bit',
 'stupid',
 'maybe',
 'not',
 'I',
 'know',
 'that',
 'Lars',
 'likes',
 'to',
 'experiment

In [ ]:
tokenizer = tfds.deprecated.text.Tokenizer()
vocabulary_set = set()
MAX_TOKENS=0

for example,label in imdb_train:
  some_token=tokenizer.tokenize(example.numpy())
  if MAX_TOKENS < len(some_token):
    MAX_TOKENS=len(some_token)
  vocabulary_set.update(some_token)

In [ ]:
vocabulary_set

{'semitic',
 'Gokbakar',
 'OTH',
 'foreclosure',
 'Adolphe',
 'adhered',
 'Empahh',
 'Damen',
 'press',
 'Heurtebise',
 'Trotter',
 'FANTASTICFANTASTICFANTASTIC',
 'Confused',
 'humanities',
 'Blake',
 'Kitt',
 'SELLOUT',
 'Ebola',
 'desecrated',
 'burnish',
 'traveller',
 'OMEGA',
 'mystical',
 'USSR',
 'underling',
 'massacring',
 'NIGHT',
 'Baron',
 'Ohs',
 'PAMELA',
 'balance',
 'eightiesly',
 'oats',
 'Kinetescope',
 'vestigial',
 'Mattter',
 'Riggs',
 'repudiated',
 'eloquent',
 'spicing',
 'muscical',
 'Muto',
 'wondering',
 'Predictably',
 'Silvio',
 'GONG',
 'EARTHQUAKE',
 'reconciles',
 'mesmerizingly',
 'mitigating',
 'renter',
 'Hancock',
 'Whispering',
 'Ennui',
 'Shaughnessy',
 'finalize',
 'whitehouse',
 'RajShree',
 'cloudy',
 'overdubbing',
 'server',
 'playbook',
 'Cassius',
 'Knighteley',
 'starts',
 'educating',
 'demoralize',
 'mapped',
 'Twist',
 'unheard',
 'Joh',
 'Apophis',
 'Rugby',
 'Eaker',
 'propagandized',
 'neglect',
 'Hallan',
 '1907',
 'attain',
 'libel

In [ ]:
type(vocabulary_set)

set

In [ ]:
len(vocabulary_set)

93929

In [ ]:
encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set, tokenizer=tokenizer)
sample_string = "Hello Dear How are you"
              
enocoded_string=encoder.encode(sample_string)
print('Encoded string is {}',format(enocoded_string))
                                                
orginal_string=encoder.decode(enocoded_string)
print('Orginal string is {}',format(orginal_string))


Encoded string is {} [59225, 64673, 65323, 61821, 34800]
Orginal string is {} Hello Dear How are you


In [ ]:
imbd_encoder= encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set, tokenizer=tokenizer)

vocab_size=imbd_encoder.vocab_size

print(vocab_size,MAX_TOKENS)

93931 2525


In [ ]:
for example, label in imdb_train.take(1):
  print(example)
  encoded=imbd_encoder.encode(example.numpy())

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)


In [ ]:
type(encoded)

list

In [ ]:
encoded

[50387,
 26542,
 40664,
 19664,
 12798,
 65571,
 41790,
 31696,
 35955,
 61553,
 90717,
 73147,
 34709,
 8134,
 92474,
 74379,
 24010,
 75905,
 61821,
 30706,
 10651,
 86984,
 43215,
 41893,
 83097,
 35955,
 30554,
 59443,
 86697,
 90717,
 2118,
 28042,
 30554,
 30706,
 91387,
 25865,
 48574,
 14309,
 43215,
 65571,
 27738,
 58054,
 72006,
 50387,
 65571,
 20591,
 40664,
 18955,
 84087,
 74160,
 41422,
 38973,
 66749,
 50952,
 53396,
 68865,
 41853,
 82044,
 67142,
 63928,
 36876,
 34020,
 41853,
 90922,
 30554,
 45246,
 31739,
 55165,
 27365,
 58001,
 41089,
 55178,
 85107,
 67034,
 31778,
 34802,
 12508,
 19385,
 43721,
 8134,
 26542,
 64983,
 86984,
 25185,
 53396,
 49429,
 46413,
 90717,
 25185,
 65571,
 85740,
 26542,
 71067,
 55725,
 30884,
 41746,
 88660,
 9694,
 46237,
 60606,
 85740,
 51473,
 61821,
 93504,
 90718,
 43215,
 25923,
 45805,
 27738,
 90718,
 34709,
 8134,
 27738,
 26116,
 65715,
 9694,
 25865,
 20609,
 40752,
 82620,
 28234]

In [ ]:
print(imbd_encoder.decode(encoded))

This was an absolutely terrible movie Don t be lured in by Christopher Walken or Michael Ironside Both are great actors but this must simply be their worst role in history Even their great acting could not redeem this movie s ridiculous storyline This movie is an early nineties US propaganda piece The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions Maria Conchita Alonso appeared phony and her pseudo love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning I am disappointed that there are movies like this ruining actor s like Christopher Walken s good name I could barely sit through it


In [ ]:
from tensorflow.keras.preprocessing import sequence

def encode_pad_transform(sample):
  encoded = imbd_encoder.encode(sample.numpy())
  pad = sequence.pad_sequences([encoded],padding='post',maxlen=150)
  return np.array(pad[0],dtype=np.int64)

def encode_tf_fn(sample,label):
  encoded = tf.py_function(encode_pad_transform,inp=[sample],Tout =(tf.int64))
  encoded.set_shape([None])
  label.set_shape([])
  return encoded, label


In [ ]:
#test the transformation on a small subset
subset=imdb_train.take(10)
tst=subset.map(encode_tf_fn)

In [ ]:
tst

<MapDataset shapes: ((None,), ()), types: (tf.int64, tf.int64)>

In [ ]:
 for review,label in tst.take(1):
   print(review, "\n",label)
   print(imbd_encoder.decode(review))

tf.Tensor(
[50387 26542 40664 19664 12798 65571 41790 31696 35955 61553 90717 73147
 34709  8134 92474 74379 24010 75905 61821 30706 10651 86984 43215 41893
 83097 35955 30554 59443 86697 90717  2118 28042 30554 30706 91387 25865
 48574 14309 43215 65571 27738 58054 72006 50387 65571 20591 40664 18955
 84087 74160 41422 38973 66749 50952 53396 68865 41853 82044 67142 63928
 36876 34020 41853 90922 30554 45246 31739 55165 27365 58001 41089 55178
 85107 67034 31778 34802 12508 19385 43721  8134 26542 64983 86984 25185
 53396 49429 46413 90717 25185 65571 85740 26542 71067 55725 30884 41746
 88660  9694 46237 60606 85740 51473 61821 93504 90718 43215 25923 45805
 27738 90718 34709  8134 27738 26116 65715  9694 25865 20609 40752 82620
 28234     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0], shape=(150,), dtype=int64) 
 tf.Tensor(0, shape=(), dtype=int64)
This was 

In [ ]:
#tokenize.encode/pad all training
# and tetrsing data
encoded_train=imdb_train.map(encode_tf_fn,num_parallel_calls=tf.data.experimental.AUTOTUNE)
encoded_test=imdb_test.map(encode_tf_fn,num_parallel_calls=tf.data.experimental.AUTOTUNE)

#preparing the model

In [ ]:
#length of the vocabulary in chars
vocab_size=imbd_encoder.vocab_size

# The embedding dimension
embedding_dim=64

#NUmber of RNN units, positive integer, dimensionality of the output space
rnn_units=64

#batch size
BATCH_SIZE=100

In [ ]:
def build_model_lstm(vocab_size,embedding_dim,rnn_units,batch_size):
  model = tf.keras.Sequential([
                tf.keras.layers.Embedding(vocab_size,embedding_dim,mask_zero=True,batch_input_shape=[batch_size,None]),
                tf.keras.layers.LSTM(rnn_units),
                tf.keras.layers.Dense(1,activation="sigmoid")
  ])
  return model

In [ ]:
model= build_model_lstm(vocab_size=vocab_size, embedding_dim=embedding_dim,
                       rnn_units=rnn_units,batch_size=BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (100, None, 64)           6011584   
_________________________________________________________________
lstm (LSTM)                  (100, 64)                 33024     
_________________________________________________________________
dense (Dense)                (100, 1)                  65        
Total params: 6,044,673
Trainable params: 6,044,673
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.save('model.h5')

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy','Precision','Recall'])

#Model training and evalution

In [ ]:
#Prefetch for performanence
encoded_train_batched=encoded_train.batch(BATCH_SIZE).prefetch(100)

In [ ]:
model.fit(encoded_train.batch(100),epochs=10)

Epoch 1/10
250/250 [==============================] - 80s 302ms/step - loss: 0.4146 - accuracy: 0.8020 - precision: 0.7836 - recall: 0.8345
Epoch 2/10
250/250 [==============================] - 76s 302ms/step - loss: 0.1857 - accuracy: 0.9334 - precision: 0.9290 - recall: 0.9386
Epoch 3/10
250/250 [==============================] - 76s 303ms/step - loss: 0.1177 - accuracy: 0.9597 - precision: 0.9587 - recall: 0.9608
Epoch 4/10
250/250 [==============================] - 76s 303ms/step - loss: 0.0842 - accuracy: 0.9710 - precision: 0.9702 - recall: 0.9718
Epoch 5/10
250/250 [==============================] - 76s 304ms/step - loss: 0.0384 - accuracy: 0.9884 - precision: 0.9881 - recall: 0.9887
Epoch 6/10
250/250 [==============================] - 77s 308ms/step - loss: 0.0449 - accuracy: 0.9863 - precision: 0.9867 - recall: 0.9858
Epoch 7/10
250/250 [==============================] - 78s 310ms/step - loss: 0.0460 - accuracy: 0.9850 - precision: 0.9856 - recall: 0.9845
Epoch 8/10
250/250 [

In [ ]:
hisotry=model.fit(encoded_train.batch(100),epochs=5)

Epoch 1/10
250/250 [==============================] - 78s 312ms/step - loss: 0.0152 - accuracy: 0.9957 - precision: 0.9948 - recall: 0.9966
Epoch 2/10
250/250 [==============================] - 77s 308ms/step - loss: 0.0092 - accuracy: 0.9974 - precision: 0.9980 - recall: 0.9968
Epoch 3/10
250/250 [==============================] - 77s 306ms/step - loss: 0.0090 - accuracy: 0.9975 - precision: 0.9975 - recall: 0.9974
Epoch 4/10
250/250 [==============================] - 76s 304ms/step - loss: 0.0136 - accuracy: 0.9962 - precision: 0.9964 - recall: 0.9961
Epoch 5/10
250/250 [==============================] - 76s 302ms/step - loss: 0.0078 - accuracy: 0.9978 - precision: 0.9981 - recall: 0.9974
Epoch 6/10
250/250 [==============================] - 72s 286ms/step - loss: 0.0095 - accuracy: 0.9970 - precision: 0.9965 - recall: 0.9974
Epoch 7/10
250/250 [==============================] - 73s 290ms/step - loss: 0.0087 - accuracy: 0.9974 - precision: 0.9976 - recall: 0.9973
Epoch 8/10
250/250 [